In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import matplotlib.pyplot as plt
import html
import re
import random
import nltk
nltk.download('punkt')
#if there is a problem in importing transformer
# please install using pip from the code below
import requests
from bs4 import BeautifulSoup
import os.path
from dateutil import parser
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
url = input("Enter the url:\n")

Enter the url:
https://arstechnica.com/gadgets/2021/07/morgan-stanley-discloses-data-breach-that-resulted-from-accellion-fta-hacks/


In [4]:
# !pip install trafilatura
# !pip install transformers
# !pip install sentencepiece

In [5]:
import trafilatura
downloaded = trafilatura.fetch_url(url)
trafilatura.extract(downloaded)
# outputs main content and comments as plain text ...
from trafilatura import bare_extraction
bare_extraction(downloaded)['title']

'Morgan Stanley discloses data breach that resulted from Accellion FTA hacks'

In [6]:
#Calculating the number of questions as needed depending upon the length of text
import math
mohack = []
text = bare_extraction(downloaded)['text']
mohack=text.splitlines()
number_of_threads = math.trunc(len(text)/3200)
if number_of_threads == 0:
  number_of_threads += 1
number_of_threads

1

In [7]:
!git clone https://github.com/amontgomerie/question_generator

fatal: destination path 'question_generator' already exists and is not an empty directory.


In [8]:
%cd question_generator/

/content/question_generator


In [9]:
from questiongenerator import QuestionGenerator
qg = QuestionGenerator()

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
lp = qg.generate(text, num_questions=number_of_threads,answer_style='sentences')

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Evaluating QA pairs...



In [12]:
lp

[{'answer': '</s> Guidehouse representatives didn’t immediately respond to an email asking why it took so long for the company to discover the breach, notify customers, and discover if other Guidehouse customers were also compromised.',
  'question': 'How long did it take for the company to find out about the breach?'}]

In [13]:
for i in range(number_of_threads):
  mohack.append(lp[i]['question'])

import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
#> '3.6.0'

3.6.0


In [14]:
mohack.append(bare_extraction(downloaded)['title'])
number_of_threads += 1

In [15]:
lp.append({"question": bare_extraction(downloaded)['title']})

In [16]:
lp

[{'answer': '</s> Guidehouse representatives didn’t immediately respond to an email asking why it took so long for the company to discover the breach, notify customers, and discover if other Guidehouse customers were also compromised.',
  'question': 'How long did it take for the company to find out about the breach?'},
 {'question': 'Morgan Stanley discloses data breach that resulted from Accellion FTA hacks'}]

In [17]:

fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [18]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in mohack])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

In [19]:
sentences = []
for i in range(len(mohack)):
  sentences.append(dictionary.doc2bow(simple_preprocess(mohack[i])))
print(softcossim(sentences[0], sentences[1], similarity_matrix))

0.6276910053845753


In [20]:
import numpy as np
import pandas as pd

def create_soft_cossim_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

matrix =create_soft_cossim_matrix(sentences)

In [21]:
matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.00,0.63,0.18,0.20,0.63,0.36,0.57,0.00,0.46,0.53,0.00,0.45,0.47,0.50,0.00,0.59,0.49,0.47,0.48,0.41,0.53
1,0.63,1.00,0.15,0.13,0.73,0.45,0.62,0.00,0.62,0.69,0.02,0.56,0.57,0.65,0.00,0.67,0.61,0.64,0.60,0.56,0.37
2,0.18,0.15,1.00,0.00,0.24,0.45,0.18,0.00,0.16,0.22,0.00,0.12,0.16,0.14,0.00,0.18,0.14,0.13,0.21,0.49,0.07
3,0.20,0.13,0.00,1.00,0.20,0.00,0.00,0.00,0.03,0.02,0.13,0.02,0.02,0.02,0.00,0.02,0.00,0.02,0.20,0.00,0.43
4,0.63,0.73,0.24,0.20,1.00,0.60,0.71,0.00,0.65,0.74,0.00,0.60,0.65,0.66,0.03,0.75,0.75,0.72,0.64,0.63,0.47
5,0.36,0.45,0.45,0.00,0.60,1.00,0.57,0.00,0.48,0.45,0.00,0.41,0.45,0.45,0.00,0.48,0.47,0.50,0.51,0.57,0.19
6,0.57,0.62,0.18,0.00,0.71,0.57,1.00,0.00,0.59,0.60,0.00,0.70,0.62,0.57,0.00,0.57,0.65,0.67,0.63,0.55,0.30
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.08,0.00,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.46,0.62,0.16,0.03,0.65,0.48,0.59,0.08,1.00,0.64,0.12,0.58,0.61,0.63,0.00,0.62,0.60,0.62,0.51,0.59,0.30
9,0.53,0.69,0.22,0.02,0.74,0.45,0.60,0.00,0.64,1.00,0.05,0.58,0.61,0.65,0.00,0.74,0.72,0.70,0.50,0.55,0.31


In [22]:
#now we will extract the paragraphs which are related to question and then 
#generate thier summary 
context = {}
for i in range(1,number_of_threads + 1):
  lm = []
  for j in range(0,len(matrix.iloc[-i])-number_of_threads):
    #only strong similarity connection paragraphs would be appended
    if matrix.iloc[-i][j] >= 0.53 and matrix.iloc[-i][j] < 1.00 :
      lm.append(j) 
    context[lp[-i]['question']] = lm 
#context
m = []
for i,j in context.items():
  if j == []:
    m.append(i)
for k in range(len(m)):
  context.pop(m[k])
context

{'How long did it take for the company to find out about the breach?': [1,
  4,
  5,
  6,
  8,
  9,
  13,
  15,
  16,
  17],
 'Morgan Stanley discloses data breach that resulted from Accellion FTA hacks': [0]}

In [23]:
from transformers import BartTokenizer, BartModel
import torch
#for the process of abstractive summmarization I have used Bart-cnn from hugging face as pretrained model
from typing import List
#bart-cnn uses encoder and decoder for summarization
#but it's only effective for tokens less than 1300 around
from transformers import BartForConditionalGeneration
def old_summarization_pipeline(text: List[str]) -> List[str]:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, truncation=True, padding=True, return_tensors='pt', max_length=1024)['input_ids']
    summary_ids = model.generate(input_ids)
    summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=False) for s in summary_ids]
    return summaries

In [24]:
for i,j in context.items():
  lm = [] 
  for k in range(len(j)):
    lm.append(old_summarization_pipeline([mohack[j[k]]]))
    context[i] = lm


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [25]:
import re
twitterthread = {}
for i,j in context.items():
 finalsummary = []
 lmo = []
 for k in range(len(j)):
    #print(i,j[k])
    lmo = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', j[k][0])
    new_list = [elem for elem in lmo if elem.strip()]
    finalsummary.append(new_list)
    dict1= {}
    d = 0
    for l in range(len(finalsummary)):
      for m in range(len(finalsummary[l])):
        d = d+1
        dict1[d] = finalsummary[l][m]
    twitterthread[i] = dict1 

In [26]:
twitterthread

{'How long did it take for the company to find out about the breach?': {1: 'The data obtained included names, addresses, dates of birth, Social Security numbers, and affiliated corporate company names.',
  2: 'A third-party service called Guidehouse was in possession of the data at the time.',
  3: 'Unknown hackers obtained the data by exploiting a series of hacks that came to light in December and January.',
  4: 'The Accellion FTA vulnerability that led to this incident was patched in January 2021, within 5 days of the patch becoming available.',
  5: 'Although the data was obtained by the unauthorized individual around that time, the vendor did not discover the attack until March of 2021.',
  6: 'Guidehouse has informed Morgan Stanley that it found no evidence that Morgan Stanley’s data had been distributed beyond the threat actor.',
  7: "Guidehouse representatives didn't immediately respond to an email asking why it took so long for the company to discover the breach, notify custo